In [ ]:
import os
import glob

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from tqdm.notebook import tqdm_notebook

os.environ["OPENCV_IO_MAX_result_PIXELS"] = pow(2,40).__str__()

import cv2

from stardist.models import StarDist2D
# prints a list of available models
StarDist2D.from_pretrained()

from stardist.plot import render_label
from csbdeep.utils import normalize
import matplotlib.pyplot as plt


In [ ]:

import glob
import cv2
import numpy as np
from stardist.models import StarDist2D
from csbdeep.utils import normalize

# Get the list of image paths
multiplex_file_paths = glob.glob("./data/multiplex/processed_32/*DAPI*.tif")

# Load the pretrained StarDist2D model
model = StarDist2D.from_pretrained('2D_versatile_fluo')

# Read the image
img = cv2.imread(multiplex_file_paths[0], 0)

# Apply Gaussian blur
blurred_img = cv2.GaussianBlur(img, (5, 5), 0)

# # Apply Otsu's thresholding
#_, otsu_thresh = cv2.threshold(blurred_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Use the thresholded image for StarDist prediction
multiplex_labels, _ = model.predict_instances_big(normalize(blurred_img), axes='YX', block_size=1024, min_overlap=128, context=128, scale=8, n_tiles=(4,4), prob_thresh=0.8)


In [ ]:
# Display the labeled image
plt.figure(figsize=(20,20))
plt.title('Florescent Image Cell count')
plt.imshow(multiplex_labels[0:1500, :])
plt.show()


In [ ]:
from skimage.measure import regionprops_table
import pandas as pd

props = regionprops_table(multiplex_labels, img, 
                          properties=['label',
                                      'area', 'equivalent_diameter',
                                      'mean_intensity', 'solidity'])

analysis_results = pd.DataFrame(props)
print(analysis_results.tail())

In [ ]:
he_file_path = glob.glob("./data/66-4/processed_32/*025*.tif")
he_img = cv2.imread(he_file_path[0])

model = StarDist2D.from_pretrained('2D_versatile_he')

he_labels, _ = model.predict_instances_big(normalize(he_img), axes='YXC',block_size=1024, min_overlap=128, context=128, scale=8, n_tiles=(4,4,1))

plt.figure(figsize=(20, 20))
plt.imshow(render_label(he_labels))
plt.axis("off")
plt.title("prediction + input overlay")

In [ ]:
props = regionprops_table(he_labels, he_img, 
                          properties=['label',
                                      'area', 'equivalent_diameter',
                                      'mean_intensity', 'solidity'])

analysis_results = pd.DataFrame(props)
print(analysis_results.tail())

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(20,20))

ax1.imshow(he_labels)
ax1.axis("off")
ax1.legend([f'Cell count {len(analysis_results)}'], fontsize=100, loc='upper right')
print(he_labels.shape)

ax2.imshow(multiplex_labels[0:1575, :])
ax2.legend([f'Cell count {len(analysis_results)}'], fontsize=100, loc='upper right')
ax2.axis("off")
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize

# Assuming he_img and img are already loaded images
# Assuming he_labels and multiplex_labels are already loaded label arrays

# Resize the labels to match the dimensions of the images
he_labels_resized = resize(np.where(he_labels >= 1, 1, 0), (1000, 1300), anti_aliasing=True)
multiplex_labels_resized = resize(np.where(multiplex_labels[0:1575, :] >= 1, 1, 0), (1000, 1300), anti_aliasing=True)

# Resize the images to match the dimensions
he_img_resized = resize(he_img, (1000, 1300), anti_aliasing=True)
img_resized = resize(img[0:1575, :], (1000, 1300), anti_aliasing=True)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

# Plot the resized images with overlaid labels
im1 = ax1.imshow(he_img_resized)
im1_overlay = ax1.imshow(he_labels_resized, cmap='Reds', alpha=0.7)
ax1.axis("off")

im2 = ax2.imshow(img_resized)
im2_overlay = ax2.imshow(multiplex_labels_resized, cmap='Blues', alpha=0.5)
ax2.axis("off")

# Annotate text on the first image
ax1.annotate('Cell Count: 5040', xy=(0, 0.95), xytext=(20, -20),
             xycoords='axes fraction', textcoords='offset points',
             bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
             fontsize=16)  # Set fontsize to 16

# Annotate text on the second image
ax2.annotate('Cell Count: 4194', xy=(0, 0.95), xytext=(20, -20),
             xycoords='axes fraction', textcoords='offset points',
             bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
             fontsize=16)  # Set fontsize to 16

plt.tight_layout()
plt.show()

# H&E cell counting

In [ ]:
he_file_path = glob.glob("./data/66-4/processed_4/*025*.tif")
he_img = cv2.imread(he_file_path[0])

model = StarDist2D.from_pretrained('2D_versatile_he')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from tifffile import imread, imsave
from csbdeep.utils import Path, normalize
from csbdeep.utils.tf import keras_import
keras = keras_import()

from stardist import export_imagej_rois, random_label_cmap
from stardist.models import StarDist2D

np.random.seed(0)
cmap = random_label_cmap()

In [ ]:
def show_image(img, **kwargs):
    """Plot large image at different resolutions."""
    fig, ax = plt.subplots(2,4, figsize=(16,8))
    mid = [s//2 + 600 for s in img.shape[:2]]
    for a,t,u in zip(ax.ravel(),[1,2,4,8,16,32,64,128],[16,8,4,2,1,1,1,1]):
        sl = tuple(slice(c - s//t//2, c + s//t//2, u) for s,c in zip(img.shape[:2],mid))
        a.imshow(img[sl], **kwargs)
        a.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
show_image(he_img)